# Kevin Oakley's       BIOF509       Spring 2019      Final Project

### Predicting Bee trends from USDA census data 

In [ ]:
## Store USDA csv files in GitHub repo in the next cell

https://github.com/ContoakleyK/BIOF509-Bee-Trends.git

In [1]:
### Import packages to pull csv file from repo for analysis

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import requests
from urllib.request import urlretrieve

In [2]:
### Assign URL to variable: url
url = 'https://raw.githubusercontent.com/ContoakleyK/BIOF509-Bee-Trends/master/USDA%20Honey%20Bee%20Colony%20Data.csv'

In [3]:
### Apply pandas package to read the .csv file: url
df = pd.read_csv(url)

In [4]:
print(df.shape)

(28463, 1)


In [5]:
df.head()


71AA6DED-1C0F-3FBB-AC96-0C7D1414549D
Program Year Period     Week Ending Geo Level State   State ANSI Ag District Ag District Code County  County ANSI Zip Code Region watershed_code Watershed Commodity Data Item                                          Domain Domain Category Value                               CV (%)
CENSUS  2012 END OF DEC NaN         COUNTY    ALABAMA 1          BLACK BELT  40               AUTAUGA 1           NaN      NaN    0              NaN       HONEY     HONEY, BEE COLONIES - INVENTORY, MEASURED IN CO... TOTAL  NOT SPECIFIED   119                                   27.7
                                                                                                                                                                     HONEY, BEE COLONIES - OPERATIONS WITH INVENTORY    TOTAL  NOT SPECIFIED   7                                     27.2
                                                                                              DALLAS  47          NaN      NaN    0              NaN       HONEY     HONEY, BEE COLONIES - INVENTORY, MEASURED IN CO... TOTAL  NOT SPECIFIED   65                                    27.7
                                                                                                                                                                     HONEY, BEE COLONIES - OPERATIONS WITH INVENTORY    TOTAL  NOT SPECIFIED   13                                    27.2

In [6]:
print(df.describe())

       71AA6DED-1C0F-3FBB-AC96-0C7D1414549D
count                                  5625
unique                                  159
top                                     (D)
freq                                    665


In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 28463 entries, (Program, Year, Period, Week Ending, Geo Level, State, State ANSI, Ag District, Ag District Code, County, County ANSI, Zip Code, Region, watershed_code, Watershed, Commodity, Data Item, Domain, Domain Category, Value) to (SURVEY, 1987, MARKETING YEAR, nan, STATE, WYOMING, 56, nan, nan, nan, nan, nan, nan, 0, nan, HONEY, HONEY, BEE COLONIES - INVENTORY, MEASURED IN COLONIES, TOTAL, NOT SPECIFIED, 39,000)
Data columns (total 1 columns):
71AA6DED-1C0F-3FBB-AC96-0C7D1414549D    5625 non-null object
dtypes: object(1)
memory usage: 900.2+ KB
None


In [8]:
### read in the smaller dataset of only Commodity = 'Honey' query
Honey_url = 'https://raw.githubusercontent.com/ContoakleyK/BIOF509-Bee-Trends/master/USDA%20Honey%20Data.csv'
honey_df = pd.read_csv(Honey_url)
print(honey_df.shape)

honey_df.info()

(45703, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45703 entries, 0 to 45702
Data columns (total 21 columns):
Program             45703 non-null object
Year                45703 non-null int64
Period              45703 non-null object
Week Ending         0 non-null float64
Geo Level           45703 non-null object
State               45703 non-null object
State ANSI          44952 non-null float64
Ag District         30394 non-null object
Ag District Code    30394 non-null float64
County              30394 non-null object
County ANSI         30359 non-null float64
Zip Code            0 non-null float64
Region              0 non-null float64
watershed_code      45703 non-null int64
Watershed           0 non-null float64
Commodity           45703 non-null object
Data Item           45703 non-null object
Domain              45703 non-null object
Domain Category     45703 non-null object
Value               45703 non-null object
CV (%)              14001 non-null object
dtypes: 

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,9,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
honey_df.head(5)

,Program,Year,Period,Week Ending,Geo Level,State,State ANSI,Ag District,Ag District Code,County,...,Zip Code,Region,watershed_code,Watershed,Commodity,Data Item,Domain,Domain Category,Value,CV (%)
0,CENSUS,2012,YEAR,NaN,COUNTY,ALABAMA,1.0,BLACK BELT,40.0,AUTAUGA,...,NaN,NaN,0,NaN,HONEY,HONEY - OPERATIONS WITH PRODUCTION,TOTAL,NOT SPECIFIED,6,33.8
1,CENSUS,2012,YEAR,NaN,COUNTY,ALABAMA,1.0,BLACK BELT,40.0,AUTAUGA,...,NaN,NaN,0,NaN,HONEY,HONEY - OPERATIONS WITH SALES,TOTAL,NOT SPECIFIED,5,32.9
2,CENSUS,2012,YEAR,NaN,COUNTY,ALABAMA,1.0,BLACK BELT,40.0,AUTAUGA,...,NaN,NaN,0,NaN,HONEY,"HONEY - SALES, MEASURED IN $",TOTAL,NOT SPECIFIED,"15,000",13.6
3,CENSUS,2012,YEAR,NaN,COUNTY,ALABAMA,1.0,BLACK BELT,40.0,ELMORE,...,NaN,NaN,0,NaN,HONEY,HONEY - OPERATIONS WITH PRODUCTION,TOTAL,NOT SPECIFIED,15,33.8
4,CENSUS,2012,YEAR,NaN,COUNTY,ALABAMA,1.0,BLACK BELT,40.0,ELMORE,...,NaN,NaN,0,NaN,HONEY,HONEY - OPERATIONS WITH SALES,TOTAL,NOT SPECIFIED,13,32.9


In [21]:
# hon is the list (np.ndarray) of column 'Commodity' to prove that the query returned only data with commodity "Honey"
hon = honey_df['Commodity'].unique()
print(hon)
print(hon.shape)

['HONEY']
(1,)


In [10]:
# lsdi is the list (np.ndarray) of column 'Data Item' to see the various sub-categories of the commodity "Honey" from my query search 
lsdi = honey_df['Data Item'].unique()
print(lsdi)
print(lsdi.shape)

['HONEY - OPERATIONS WITH PRODUCTION' 'HONEY - OPERATIONS WITH SALES'
 'HONEY - SALES, MEASURED IN $'
 'HONEY, BEE COLONIES - INVENTORY, MEASURED IN COLONIES'
 'HONEY, BEE COLONIES - OPERATIONS WITH INVENTORY'
 'HONEY, BEE COLONIES - OPERATIONS WITH SALES'
 'HONEY, BEE COLONIES - SALES, MEASURED IN COLONIES'
 'HONEY, BEE COLONIES - ADDED & REPLACED, MEASURED IN COLONIES'
 'HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DISORDER, MEASURED IN COLONIES'
 'HONEY, BEE COLONIES - LOSS, DEADOUT, MEASURED IN COLONIES'
 'HONEY, BEE COLONIES - LOSS, DEADOUT, MEASURED IN PCT OF COLONIES'
 'HONEY, BEE COLONIES, AFFECTED BY DISEASE - INVENTORY, MEASURED IN PCT OF COLONIES'
 'HONEY, BEE COLONIES, AFFECTED BY OTHER CAUSES - INVENTORY, MEASURED IN PCT OF COLONIES'
 'HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - INVENTORY, MEASURED IN PCT OF COLONIES'
 'HONEY, BEE COLONIES, AFFECTED BY PESTS ((EXCL VARROA MITES)) - INVENTORY, MEASURED IN PCT OF COLONIES'
 'HONEY, BEE COLONIES, AFFECTED BY UNKNOWN CAUSES -

In [11]:
# years is the numpy.ndarray for the unique entries of year associated throughout the honey_df DataFrame
years = honey_df['Year'].unique()
print(years)
print(years.shape)

[2012 2007 2002 1997 2018 2017 2016 2015 2014 2013 2011 2010 2009 2008
 2006 2005 2004 2003 2001 2000 1999 1998 1996 1995 1994 1993 1992 1991
 1990 1989 1988 1987]
(32,)


In [12]:
# period is the np.ndarray of time designations for entries
period = honey_df['Period'].unique()
print(period)
print(period.shape)

['YEAR' 'END OF DEC' 'FIRST OF JAN' 'JAN THRU MAR' 'APR THRU JUN'
 'FIRST OF APR' 'MARKETING YEAR' 'FIRST OF JUL' 'JUL THRU SEP'
 'FIRST OF OCT' 'OCT THRU DEC' 'MID DEC']
(12,)


In [13]:
# census is the np.ndarray for the 'census' and 'survey' designations of the 'Program' column
census = honey_df['Program'].unique()
print(census)
print(census.shape)

['CENSUS' 'SURVEY']
(2,)


In [14]:
# state is the numpy.ndarray for the States represented throughout the honey_df DataFrame.  Notice entries for "US TOTAL" and "OTHER STATES"
state = honey_df['State'].unique()
print(state)
print(state.shape)

['ALABAMA' 'ALASKA' 'ARIZONA' 'ARKANSAS' 'CALIFORNIA' 'COLORADO'
 'CONNECTICUT' 'DELAWARE' 'FLORIDA' 'GEORGIA' 'HAWAII' 'IDAHO' 'ILLINOIS'
 'INDIANA' 'IOWA' 'KANSAS' 'KENTUCKY' 'LOUISIANA' 'MAINE' 'MARYLAND'
 'MASSACHUSETTS' 'MICHIGAN' 'MINNESOTA' 'MISSISSIPPI' 'MISSOURI' 'MONTANA'
 'NEBRASKA' 'NEVADA' 'NEW HAMPSHIRE' 'NEW JERSEY' 'NEW MEXICO' 'NEW YORK'
 'NORTH CAROLINA' 'NORTH DAKOTA' 'OHIO' 'OKLAHOMA' 'OREGON' 'PENNSYLVANIA'
 'RHODE ISLAND' 'SOUTH CAROLINA' 'SOUTH DAKOTA' 'TENNESSEE' 'TEXAS' 'UTAH'
 'VERMONT' 'VIRGINIA' 'WASHINGTON' 'WEST VIRGINIA' 'WISCONSIN' 'WYOMING'
 'US TOTAL' 'OTHER STATES']
(52,)


In [15]:
# sansi is the numpy.ndarray for the American National Standards Institute. ANSI codes are a standardized set of numeric or alphabetic codes issued by the American National Standards Institute (ANSI) to ensure uniform identification of geographic entities through all federal government agencies. These standards replace the Federal Information Processing Standards (FIPS) codes previously issued by the National Institute of Standards and Technology (NIST). See http://www.census.gov/geo/www/ansi/ for more information.
sansi = honey_df['State ANSI'].unique()
print(sansi)
print(sansi.shape)

[ 1.  2.  4.  5.  6.  8.  9. 10. 12. 13. 15. 16. 17. 18. 19. 20. 21. 22.
 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39. 40.
 41. 42. 44. 45. 46. 47. 48. 49. 50. 51. 53. 54. 55. 56. nan]
(51,)


In [16]:
# ag_dist is the numpy.ndarray for the Agricultural Districts represented throughout the honey_df DataFrame.  
ag_dist = honey_df['Ag District'].unique()
print(ag_dist)
print(ag_dist.shape)

['BLACK BELT' 'COASTAL PLAINS & GULF COAST' 'MOUNTAINS & EASTERN VALLEY'
 'NORTHERN VALLEY' 'UPPER PLAINS & PIEDMONT' 'WIREGRASS' 'KENAI PENINSULA'
 'MATANUSKA-SUSITNA-ANCHORAGE' 'TANANA VALLEY' 'NORTHERN' 'SOUTHERN'
 'CENTRAL' 'EAST CENTRAL' 'NORTH CENTRAL' 'NORTHEAST' 'NORTHWEST'
 'SOUTH CENTRAL' 'SOUTHEAST' 'SOUTHWEST' 'WEST CENTRAL' 'CENTRAL COAST'
 'NORTHERN COAST' 'SACRAMENTO VALLEY' 'SAN JOAQUIN VALLEY'
 'SIERRA MOUNTAINS' 'SISKIYOU-SHASTA' 'SOUTHERN CALIFORNIA'
 'NORTHWEST AND MOUNTAIN' 'SAN LUIS VALLEY' 'ALL COUNTIES' 'HAWAII ISLAND'
 'KAUAI AND NIIHAU ISLANDS' 'MAUI, MOLOKAI & LANAI ISLAND' 'OAHU ISLAND'
 'EAST' 'NORTH' 'EAST SOUTHEAST' 'WEST' 'WEST SOUTHWEST' 'BLUEGRASS'
 'EASTERN OR MOUNTAIN' 'MIDWESTERN' 'PURCHASE' 'SOUTH'
 'LOWER EASTERN SHORE' 'UPPER EASTERN SHORE' 'WESTERN' 'UPPER PENINSULA'
 'LOWER DELTA' 'SOUTHEAST AND COASTAL' 'UPPER DELTA' 'EASTERN'
 'LONG ISLAND' 'CENTRAL COASTAL' 'CENTRAL PIEDMONT' 'NORTHERN COASTAL'
 'NORTHERN MOUNTAIN' 'NORTHERN PIEDMONT' 'SOUTH

In [17]:
# ag_dist_cd is the numpy.ndarray for the Agricultural District Codes represented throughout the honey_df DataFrame.  
ag_dist_cd = honey_df['Ag District Code'].unique()
print(ag_dist_cd)
print(ag_dist_cd.shape)

[40. 50. 20. 10. 30. 60. 80. 90. 70. 51. 11. 24. 35. 62. 91. 82. 52. 97.
 21. 81. 96. 22. nan 12.]
(24,)


In [18]:
# county is the numpy.ndarray for the Counties represented throughout the honey_df DataFrame.
county = honey_df['County'].unique()
print(county)
print(county.shape)

['AUTAUGA' 'ELMORE' 'GREENE' ... 'HANSFORD' 'LAMB' 'PECOS']
(1695,)


In [19]:
# cansi is the numpy.ndarray for the Countie ANSI codes represented throughout the honey_df DataFrame.
cansi = honey_df['County ANSI'].unique()
print(cansi)
print(cansi.shape)

[  1.  51.  63.  85.  87.  91. 105. 119.   3.  13.  23.  25.  35.  53.
  97.  99. 129.   9.  15.  19.  29.  43.  49.  55.  71.  95. 115.  33.
  59.  77.  79.  83.  89.  93. 103. 133.   7.  17.  21.  27.  37.  57.
  73.  75.  81. 107. 111. 117. 121. 123. 125. 127.   5.  31.  39.  41.
  45.  61.  67.  69. 109. 113.  nan  12. 147.  65. 135. 137. 141. 145.
 143.  47. 131. 149.  11. 101.  86. 153. 159. 167. 169. 175. 207. 209.
 211. 217. 225. 235. 237. 247. 265. 163. 189. 251. 139. 157. 187. 219.
 227. 281. 291. 297. 311. 181. 195. 221. 241. 257. 317. 213. 223. 233.
 295. 313. 155. 161. 173. 185. 271. 277. 287. 321. 179. 183. 229. 267.
 299. 305. 177. 201. 205. 243. 253. 261. 273. 151. 171. 197. 199. 231.
 255. 263. 269. 285. 293. 203. 191. 193. 165. 215. 239. 186. 510. 309.
 331. 349. 397. 439. 491. 409. 333. 337. 363. 367. 425. 497. 315. 343.
 347. 365. 379. 387. 401. 419. 423. 449. 459. 467. 499. 289. 339. 351.
 373. 395. 403. 407. 455. 457. 471. 473. 411. 451. 463. 485. 325. 453.
 477. 

In [20]:
# wash_cd is the numpy.ndarray for the Watershed Code represented throughout the honey_df DataFrame. Seems uninformative
wash_cd = honey_df['watershed_code'].unique()
print(wash_cd)
print(wash_cd.shape)

[0]
(1,)


In [22]:
# domain is the list (np.ndarray) of column 'Domain' to see the various sub-categories
domain = honey_df['Domain'].unique()
print(domain)
print(domain.shape)

['TOTAL' 'OPERATORS' 'OPERATORS, PRINCIPAL' 'INVENTORY OF BEE COLONIES']
(4,)


In [23]:
# value is the list (np.ndarray) of column 'Value' to see the various sub-categories
value = honey_df['Value'].unique()
print(value)
print(value.shape)

['6' '5' '15,000' ... '1,188,000' '531,000' '5,574,000']
(4542,)


In [30]:
# Start Preprocessing: Look through the honey_df for missing data in columns of interest based on the above exploration
print(honey_df.dtypes)
print(honey_df.shape)

honey_drop = honey_df.dropna(axis=1, thresh=3)
print(honey_drop.info())

Program              object
Year                  int64
Period               object
Week Ending         float64
Geo Level            object
State                object
State ANSI          float64
Ag District          object
Ag District Code    float64
County               object
County ANSI         float64
Zip Code            float64
Region              float64
watershed_code        int64
Watershed           float64
Commodity            object
Data Item            object
Domain               object
Domain Category      object
Value                object
CV (%)               object
dtype: object
(45703, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45703 entries, 0 to 45702
Data columns (total 17 columns):
Program             45703 non-null object
Year                45703 non-null int64
Period              45703 non-null object
Geo Level           45703 non-null object
State               45703 non-null object
State ANSI          44952 non-null float64
Ag District         30394

In [28]:
# Check how many values are missing in the Ag District Code column
print(honey_df['Ag District Code'].isnull().sum())

# Subset the honey dataset
honey_subset = honey_drop[honey_drop['Ag District Code'].notnull()]

# Print out the shape of the subset
print(honey_subset.shape)


15309
(30394, 17)


In [29]:
# I got a warning in the hone_df.info() output for the mix of dtypes in 'Ag District Code' column.  Looking into the column earlier I saw that there are 24 unique codes, one of which is 'nan'.
# Check how many values are missing in the 'Ag District Code' column
print(honey_subset[['Ag District Code']].isnull().sum())

# Keep only rows where length_of_time, state, and type are not null
honey_subset_no_missing = honey_subset[honey_subset["Ag District Code"].notnull()]

# Print out the shape of the new dataset
print(honey_subset_no_missing.shape)
print(honey_subset_no_missing.info())

Ag District Code    0
dtype: int64
(30394, 17)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30394 entries, 0 to 31120
Data columns (total 17 columns):
Program             30394 non-null object
Year                30394 non-null int64
Period              30394 non-null object
Geo Level           30394 non-null object
State               30394 non-null object
State ANSI          30394 non-null float64
Ag District         30394 non-null object
Ag District Code    30394 non-null float64
County              30394 non-null object
County ANSI         30359 non-null float64
watershed_code      30394 non-null int64
Commodity           30394 non-null object
Data Item           30394 non-null object
Domain              30394 non-null object
Domain Category     30394 non-null object
Value               30394 non-null object
CV (%)              13672 non-null object
dtypes: float64(3), int64(2), object(12)
memory usage: 4.2+ MB
None


In [51]:
# Change the 'Value' column entries to float64 dtypes from object dtypes
honey_df["Value"] = honey_df["Value"].astype(float)
# Check the column types
print(honey_df["Value"].dtypes)

# looking into stackoverflow to solve the ValueError I found this?  frame["time"]=frame["time"].apply(lambda x:x[0])
# but when I applied it to the current column it gave object dtype.
honey_df["Value"].head()

ValueError: setting an array element with a sequence.